In [1]:
import re
from nltk.stem import WordNetLemmatizer


def clean_text(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    words = text.lower().split()
    lemmatizer = WordNetLemmatizer()
    lem_words = [lemmatizer.lemmatize(w, pos='n') for w in words]
    lem_words = [lemmatizer.lemmatize(w, pos='v') for w in lem_words]
    stopwords = {}.fromkeys([line.rstrip() for line in open('F:\\PycharmProjects\\NVDproject\\nvdcve\\stopwords.txt')])
    eng_stopwords = set(stopwords)
    words = [w for w in lem_words if w not in eng_stopwords]
    return words

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from gensim.models import word2vec  # 导入gensim包
from tensorflow.python.keras.models import Sequential

years = ['2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022']
infix = ''
infix = str(years[0])+"to"+str(years[len(years)-1])
infix+='_no'

cwe_min_count = 3000


infix+='_'+str(cwe_min_count)

vec_len = 100
# vec_len = 200
# vec_len = 300
min_count = 1
window_len = 5
# dense_unit = 128
dense_unit = 256
wv_model_path  = '..//..//models//wv//'+infix+"_"+str(vec_len)+"_"+str(min_count)+"_"+str(window_len)+'.pkl'
label_path = '..\\..\\data\\clean\\nvdcve-1.1-'+infix+'_labels.csv'
n=30

cwe_count = 11

def get_label_one_hot(list):
    values = np.array(list)
    n_values = np.max(values) + 1
    return np.eye(n_values)[values]

from tensorflow.keras.layers import Conv1D,Conv2D, BatchNormalization, Activation, MaxPool1D,MaxPool2D, Dropout, Flatten, Dense, GRU
from tensorflow.keras import Model

In [3]:
if __name__ == '__main__':
    wv_model = word2vec.Word2Vec.load(wv_model_path)
    dataset = pd.read_csv(label_path, header=None)
    dataset.columns = ['cve_id', 'cwe_id', 'descript', 'label']
    dataset['contents'] = dataset['descript'].apply(clean_text)
    train_dataset = []
    for line in dataset['contents']:
        length = len(line)
        if length > n:
            line = line[:n]
            word2vec_matrix = (wv_model.wv[line])
            train_dataset.append(word2vec_matrix)
        else:
            word2vec_matrix = (wv_model.wv[line])
            pad_length = n - length
            pad_matrix = np.zeros([pad_length, vec_len]) + 1e-10
            word2vec_matrix = np.concatenate([word2vec_matrix, pad_matrix], axis=0)
            train_dataset.append(word2vec_matrix)
    # train_dataset = np.expand_dims(train_dataset, 3)
    label_dataset = get_label_one_hot(dataset['label'])

    from sklearn.model_selection import train_test_split
    import keras_metrics as km
    from tensorflow.keras.metrics import Precision,Recall 
    Metrics = ['accuracy',Precision(),Recall(),km.f1_score()]
    # 顺序模型
    model = Sequential()
    model.add(Conv1D(256, 3, padding='same', strides=1, activation='relu'))
    model.add(MaxPool1D(pool_size=2))
    model.add(GRU(256, return_sequences=True))
    model.add(GRU(256,dropout=0.5, recurrent_dropout=0.1))
    model.add(Dense(cwe_count,activation="softmax"))

    x_train, x_test, y_train, y_test = train_test_split(train_dataset, label_dataset, test_size=0.2, random_state=217)
    batch_size = 32
    train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)

In [4]:
model.compile(optimizer=tf.optimizers.Adam(1e-3),
                  loss=tf.losses.categorical_crossentropy, metrics=Metrics)
history = model.fit(train_data, epochs=10)
score = model.evaluate(np.array(x_test), y_test)
print('last score:', score)
print(model.summary())

Epoch 1/10


C:\Users\Lzh\miniconda3\envs\TF2.4\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:1402: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


2128/2128 [==============================] - 372s 170ms/step - loss: 0.6716 - accuracy: 0.7692 - precision: 0.8590 - recall: 0.7041 - f1_score: 0.5910
Epoch 2/10
2128/2128 [==============================] - 362s 170ms/step - loss: 0.3707 - accuracy: 0.8805 - precision: 0.9062 - recall: 0.8571 - f1_score: 0.8256
Epoch 3/10
2128/2128 [==============================] - 356s 167ms/step - loss: 0.3209 - accuracy: 0.8928 - precision: 0.9157 - recall: 0.8746 - f1_score: 0.8386 - loss: 0.3211 - accuracy: 0.8928 -
Epoch 4/10
2128/2128 [==============================] - 365s 172ms/step - loss: 0.2815 - accuracy: 0.9042 - precision: 0.9212 - recall: 0.8901 - f1_score: 0.8509
Epoch 5/10
2128/2128 [==============================] - 358s 168ms/step - loss: 0.2544 - accuracy: 0.9134 - precision: 0.9287 - recall: 0.9013 - f1_score: 0.8584
Epoch 6/10
2128/2128 [==============================] - 358s 168ms/step - loss: 0.2294 - accuracy: 0.9214 - precision: 0.9342 - recall: 0.9114 - f1_score: 0.8795
Epo